In [1]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from lxml import etree
import csv
import os
from time import sleep
from random import randint
from tqdm import *
import tushare as ts
pro = ts.pro_api('6792f89c67a1f228aaf8aa19e6b0ca381504c16d8d5b7f75d76427a5')

In [2]:
data = pro.query('stock_basic', exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date,market,exchange,list_status,is_hs')

In [3]:
stock_name = pd.DataFrame(zip(data.ts_code,data.name))
stock_name.columns = ['code','name']
stock_name.to_csv('./stock_name.csv')

In [4]:
headers = {'user-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'}

In [5]:
def parseTime(unformatedTime):
    if '分钟' in unformatedTime:
        minute = unformatedTime[:unformatedTime.find('分钟')]
        minute = timedelta(minutes=int(minute))
        return (datetime.now() -
                minute).strftime('%Y-%m-%d %H:%M')
    elif '小时' in unformatedTime:
        hour = unformatedTime[:unformatedTime.find('小时')]
        hour = timedelta(hours=int(hour))
        return (datetime.now() -
                hour).strftime('%Y-%m-%d %H:%M')
    else:
        return unformatedTime


def dealHtml(html):
    results = html.xpath('//div[@class="result-op c-container xpath-log new-pmd"]')

    saveData = []

    for result in results:
        title = result.xpath('.//h3/a')[0]
        title = title.xpath('string(.)').strip()

        summary = result.xpath('.//span[@class="c-font-normal c-color-text"]')[0]
        summary = summary.xpath('string(.)').strip()

        # ./ 是直接下级，.// 是直接/间接下级
#         infos = result.xpath('.//div[@class="news-source"]')[0]
#         source, dateTime = infos.xpath(".//span[last()-1]/text()")[0], \
#                           infos.xpath(".//span[last()]/text()")[0]
        
        try:
            infos = result.xpath('.//div[@class="news-source"]')[0]
            source, dateTime = infos.xpath(".//span[last()-1]/text()")[0], \
                              infos.xpath(".//span[last()]/text()")[0]
        except:
            continue 
        
        dateTime = parseTime(dateTime)

#         print('标题', title)
#         print('来源', source)
#         print('时间', dateTime)
#         print('概要', summary)
#         print('\n')

        saveData.append({
            'title': title,
            'source': source,
            'time': dateTime,
            'summary': summary
        })
    with open(fileName, 'a+', encoding='utf-8-sig', newline='') as f:
        writer = csv.writer(f)
        for row in saveData:
            writer.writerow([row['title'], row['source'], row['time'], row['summary']])


#headers = {
#    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36',
#    'Referer': 'https://www.baidu.com/s?rtt=1&bsst=1&cl=2&tn=news&word=%B0%D9%B6%C8%D0%C2%CE%C5&fr=zhidao'
#}
headers = headers
url = 'https://www.baidu.com/s'

params = {
    'ie': 'utf-8',
    'medium': 0,
    # rtt=4 按时间排序 rtt=1 按焦点排序
    'rtt': 1,
    'bsst': 1,
    'rsv_dl': 'news_t_sk',
    'cl': 2,
    'tn': 'news',
    'rsv_bp': 1,
    'oq': '',
    'rsv_btype': 't',
    'f': 8,
}


def doSpider(keyword,code,sortBy = 'focus'):
    '''
    :param keyword: 搜索关键词
    :param sortBy: 排序规则，可选：focus(按焦点排序），time(按时间排序），默认 focus
    :return:
    '''
    global fileName
    fileName = '{}.csv'.format(code +"_"+ keyword)
    if not os.path.exists(fileName):
        with open(fileName, 'w+', encoding='utf-8-sig', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['title', 'source', 'time', 'summary'])

    params['wd'] = keyword
    if sortBy == 'time':
        params['rtt'] = 4

    response = requests.get(url=url, params=params, headers=headers)

    html = etree.HTML(response.text)

    dealHtml(html)

    total = html.xpath('//div[@id="header_top_bar"]/span/text()')[0]

    total = total.replace(',', '')

    total = int(total[7:-1])

    pageNum = total // 10
    #pageNum = 5

    for page in range(1, pageNum):
        print('第 {} 页\n\n'.format(page))
        headers['Referer'] = response.url
        params['pn'] = page * 10

        response = requests.get(url=url, headers=headers, params=params)

        html = etree.HTML(response.text)

        dealHtml(html)

        sleep(randint(2, 4))
    ...
def single_spider(indices):
    print(indices)
    for ind in tqdm(indices):
        doSpider(keyword = stock_name['name'][ind],code = stock_name['code'][ind],sortBy='focus')
# if __name__ == "__main__":
#     for i in tqdm(range(len(stock_name))):
#         doSpider(keyword = stock_name['name'][i],code = stock_name['code'][i],sortBy='focus')

In [6]:
from multiprocessing import Pool

In [ ]:
    pnum=1
    start = 43
    pool = Pool(pnum)
    length = len(stock_name) - start
    perlength = int(np.ceil(length/pnum))
    for p in range(pnum):
        print(p)
        indices = [i for i in range(start+p*perlength, start+(p+1)*perlength)]
        pool.apply_async(single_spider, args=(indices,)) 
    pool.close()
    pool.join()

0
